In [ ]:
# nedirba cia niekas nes nera dataseto

In [1]:
# Acoustic Model

import torch
import torch.nn as nn
import torch.optim as optim

class SimpleTTSModel(nn.Module):
    def __init__(self):
        super(SimpleTTSModel, self).__init__()
        self.lstm = nn.LSTM(input_size=128, hidden_size=256, num_layers=2, batch_first=True)
        self.output_layer = nn.Linear(256, 80)  # Predicting mel-spectogram with 80 mel bands

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.output_layer(x)
        return x

In [ ]:
# Training loop

# Assuming you have a DataLoader providing input sequences and target mel-spectograms
model = SimpleTTSModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):  # Runs 50 epochs
    for i, (input_sequences, target_spectograms) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(input_sequences)
        loss = criterion(outputs, target_spectograms)
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch+1}/50], Loss: {loss.item():.4f}')


In [ ]:
# Generating speech

import librosa
import numpy as np

def griffin_lim(spectogram):
    angles = np.exp(2j * np.pi * np.random.rand(*spectogram.shape))
    S_complex = np.abs(spectogram).astype(np.complex)
    y = librosa.istft(S_complex * angles)
    for _ in range(100):  # Griffin-Lim iterations
        angles = np.exp(1j * np.angle(librosa.stft(y, n_fft=2048)))
        y = librosa.istft(S_complex * angles)
    return y
